In [1]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
tabelas = pd.read_html("https://mudancanorteblumenau.paginas.ufsc.br/logistica/area-dos-lideres-de-ambiente/lista-de-ambientes-e-lideres-designados/")

In [3]:
DADOS = []

for tabela in tabelas:
    edificio = tabela.iloc[0,0]
    cabecalho = tabela.iloc[1,:]
    dados = tabela.iloc[2:,:]
    dados.columns = cabecalho
    dados["edificio"] = edificio
    DADOS.append(dados)

/tmp/ipykernel_1753190/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1753190/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1753190/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [4]:
DADOS = pd.concat(DADOS)

In [5]:
def trysplit(string: str, n: int):
    try:
        return string.split(" – ")[n].strip()
    except IndexError:
        return trysplit(string, n-1)

In [6]:
DADOS["a_codigo"] = DADOS.Ambiente.apply(lambda x: trysplit(x,0))
DADOS["a_nome"] = DADOS.Ambiente.apply(lambda x: trysplit(x,1))

In [7]:
DADOS

1,Ambiente,Líder de Ambiente,edificio,a_codigo,a_nome
2,A001 – BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU ...,Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo)
3,A001.1 – ADMINISTRAÇÃO BIBLIOTECA (Térreo),Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BIBLIOTECA (Térreo)
4,A001.3 – AGENTE PATRIMONIAL (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo)
5,A001.3.1 – SALA DO SERVIDOR (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo)
6,A002 – SECRETARIA ACADÊMICA (Térreo),Cláudio de Morais,BNU02 – Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo)
...,...,...,...,...,...
65,CC01 – CORREDOR 1º ANDAR (1º Andar),Patrícia A. A. de F. Barthel,BNU07 – Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar)
66,CC02 – CORREDOR 2º ANDAR (2º Andar),Marilena Corrêa Fialho,BNU07 – Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar)
67,CC03 – CORREDOR 3º ANDAR (3º Andar),Ivan de Matos,BNU07 – Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar)
2,RU001 – Restaurante Universitário (Térreo),Edgar Noschang Kunz,BNU08 – Restaurante Universitário,RU001,Restaurante Universitário (Térreo)


In [8]:
DADOS.rename(columns={"Líder de Ambiente": "lider"}, inplace=True)

In [9]:
lideres = DADOS.lider.unique()

In [10]:
DATABASE = {}

for lider in lideres:
    DATABASE[lider] = list(DADOS.loc[DADOS.lider == lider, "a_codigo"]) 

In [11]:
import json

#with open("json/lideres.json", "w") as f:
#    json.dump(DATABASE, f, indent=2, ensure_ascii=False)

In [12]:
full_ambientes = DADOS.a_codigo.unique()

In [13]:
from load_lideres import load_json_file

In [14]:
ambientes = pd.DataFrame(load_json_file("json/ambientes.json"))

In [15]:
JOIN = pd.merge(left=DADOS,right=ambientes, left_on="a_codigo", right_on="codigo", how="outer")

In [16]:
JOIN

,Ambiente,lider_x,edificio_x,a_codigo,a_nome,edificio_y,codigo,nome,items,lider_y,status,campus
0,A001 – BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU ...,Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo),BNU02 - Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo),256,None,Em uso,Sul
1,A001.1 – ADMINISTRAÇÃO BIBLIOTECA (Térreo),Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BIBLIOTECA (Térreo),BNU02 - Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BILBIOTECA (Térreo),27,None,Em uso,Sul
2,A001.3 – AGENTE PATRIMONIAL (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo),BNU02 - Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo),8,None,Em uso,Sul
3,A001.3.1 – SALA DO SERVIDOR (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo),BNU02 - Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo),4,None,Em uso,Sul
4,A002 – SECRETARIA ACADÊMICA (Térreo),Cláudio de Morais,BNU02 – Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo),BNU02 - Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo),58,None,Em uso,Sul
...,...,...,...,...,...,...,...,...,...,...,...,...
182,CC01 – CORREDOR 1º ANDAR (1º Andar),Patrícia A. A. de F. Barthel,BNU07 – Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar),BNU07 - Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar),6,None,Em uso,Sul
183,CC02 – CORREDOR 2º ANDAR (2º Andar),Marilena Corrêa Fialho,BNU07 – Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar),BNU07 - Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar),5,None,Em uso,Sul
184,CC03 – CORREDOR 3º ANDAR (3º Andar),Ivan de Matos,BNU07 – Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar),BNU07 - Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar),5,None,Em uso,Sul
185,RU001 – Restaurante Universitário (Térreo),Edgar Noschang Kunz,BNU08 – Restaurante Universitário,RU001,Restaurante Universitário (Térreo),BNU08 - Restaurante Universitário,RU001,Restaurante Universitário (Térreo),5,None,Em uso,Sul


In [17]:
AMBIENTES_VAZIOS = []
for row in JOIN[JOIN.codigo.isna()].itertuples(): 
    dados = {
    "edificio":row.edificio_x.replace("–", "-"),
    "codigo": row.a_codigo,
    "nome":row.a_nome,
    "items": 0,
    "lider": None,
    "status":"Em uso",
    "campus":"Sul"
  }
    AMBIENTES_VAZIOS.append(dados)

In [18]:
ambientes_vazios = pd.DataFrame(AMBIENTES_VAZIOS)

In [19]:
ambientes_vazios

""


In [20]:
full_ambientes  = pd.concat([ambientes, ambientes_vazios]).sort_values(by="codigo")


In [21]:
full_ambientes.to_json("json/ambientes.json",  orient="records", indent=2)

In [47]:
lideres = load_json_file("json/lideres.json")

In [42]:
contatos_taes = pd.read_html("https://servidores.blumenau.ufsc.br/servidores-tae/")[0]
contatos_taes.rename(columns={"E-mail Chat UFSC": "email", "Servidores": "nome"}, inplace=True)
contatos_taes.email = contatos_taes.email.str.extract(r"(\S*@ufsc.br)")

In [43]:
contatos_profs = pd.concat(pd.read_html("https://servidores.blumenau.ufsc.br/servidores-docentes/"))
contatos_profs.rename(columns={"Professores": 'nome', "E-mail": 'email'}, inplace=True)

In [46]:
nomes_emails = pd.concat([contatos_taes, contatos_profs])[["nome", "email"]]

In [53]:
def userModel(nome, ambientes):
    return {
        "nome": nome, 
        "role": "Líder de Ambiente",
        "usuario_de": [],
        "ambientes": ambientes
    }

In [58]:
LIDERES = {}
for nome, ambientes in lideres.items():
    pesquisa = nomes_emails.query(f"nome == '{nome}'")
    n, m = pesquisa.shape
    if n == 1 and m == 2:
        LIDERES[pesquisa.email.values[0]] = userModel(nome, ambientes)
    else: 
        LIDERES[nome] = userModel(nome, ambientes)


In [60]:
import json

with open("json/lideres.json", "w") as f:
    json.dump(LIDERES, f, indent=2, ensure_ascii=False)